# Sleep Stage Classification: Analyzing EEG Date for Sleep Patterns

This model takes a look at the sleep stages based on brain activity patterns recorded using EEG(Electroencephalography). This aids in diagnosing sleep disorders as well as eventually helping to improve sleep quality through understanding a person's sleep patterns and common sleep stages.

Sleep can be divided into numerous stages; Awake: rapid, low-amplitude EEG activity, REM(Rapid Eye Movement): vivid dreams and unique EEG patterns, and NREM(Non-Rapid Eye Movement), which can be subsetted into light sleep, deeper sleep and deep sleep categorized by moderate EEG activity for light sleep and slow-waves for deep and deeper sleep.

# Dataset and Methodology

For this project we used the PhysioNet Sleep Dataset which contains the EEG recordings from participants over numerous nights. Each recording is labeled with the corresponding sleep stages aforementioned. This dataset includes several hours of continuous EEG recordings as well as annotations of each 30 second window of EEG data which is where the sleep is labeled as one of the stages.

We explored various algorithms to classify sleep stages such as KNN and Logistic Regression. Midway into the project we thought about using random forest as it is effective in handling complex data structures but in the end we went with KNN and Logistic Regression as it was a balanced trade off in performance. Though we may attempt it again as we believe their might be a slight increase as we near are increasing accuracy in the KNN and logistic regression models.

In [1]:
!pip install mne==1.8.0

In [5]:
import mne

# Load EEG data from EDF file
# Use the correct file path format for the current environment.
data = mne.io.read_raw_edf('/datasets/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7011J0-PSG.edf', preload=True)
annotations = mne.read_annotations('/datasets/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4001EC-Hypnogram.edf')
data.set_annotations(annotations)

In [ ]:
# Extract sleep stage annotations
labels = []
for annot in annotations:
    if 'Sleep stage' in annot['description']:
        stage = annot['description'][-1]
        labels.append(stage)

# Define a mapping dictionary for sleep stages
stage_mapping = {'1': 'Light Sleep', '2': 'Deeper Sleep', '3': 'Deep Sleep', 'R': 'REM', 'W': 'Awake'}
labels = [stage_mapping.get(stage, 'Unknown') for stage in labels]

# Pre-Processing

In [ ]:
# Filter EEG data (bandpass filter for sleep frequency bands)
data.filter(0.5, 49.5)
eeg_data = data.get_data(picks=['eeg'])

# Split into windows
window_size = int(30 * data.info['sfreq'])  # 30-second windows
windows = [eeg_data[:, i:i + window_size] for i in range(0, len(eeg_data[0]), window_size)]
windows = np.array([win for win in windows if win.shape[1] == window_size])

def extract_features(window):
    """ Extract power spectral density features from a window of EEG data """
    freqs, psd = welch(window, fs=data.info['sfreq'], nperseg=256)
    return np.log(psd.mean(axis=1))  # Take log mean across channels for each frequency bin

# Extract features for each window
features = np.array([extract_features(win) for win in windows])

# Map labels to numeric values
label_mapping = {'Light Sleep': 0, 'Deeper Sleep': 1, 'Deep Sleep': 2, 'REM': 3, 'Awake': 4}

# Exploratory Data Analysis 

<img src="LR.jpg.png" width="" align="" />

<img src="KN.jpg.png" width="" align="" />

This dataset contains labels for each sleep stage. Through EDA, we are able to visualize the distribution to understand the balance among the stages. As observed here, we can see that there are more light and deep sleep stages which required more weighting and data augmenting techniques.

We also realized that both models had their ups and downs; for example KNN performs well on light sleep but is not the best at differentiating REM, whereas Logistic Regression is more balanced all around but the accuracy for Deep Sleep isn't the best. This highlight the issue and challenge of distinguishing between the similar stages of light, deeper and deep sleep in both models.

In [19]:
valid_labels = [label for label in labels if label in label_mapping]
numeric_labels = [label_mapping[label] for label in valid_labels]

# Ensure the number of labels matches the number of windows
if len(windows) != len(numeric_labels):
    min_length = min(len(windows), len(numeric_labels))
    windows = windows[:min_length]
    numeric_labels = numeric_labels[:min_length]

# Ensure that both features and labels have the same length
features_length = len(features)
labels_length = len(numeric_labels)

print(f"Features length: {features_length}, Labels length: {labels_length}")

if features_length != labels_length:
    min_len = min(features_length, labels_length)
    features = features[:min_len]
    numeric_labels = numeric_labels[:min_len]

In [30]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(features, numeric_labels, test_size=0.2, stratify=numeric_labels)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

We split the dataset into training and testing sets to evaluate model performance on unseen data. Stratifying by numeric_labels ensures that each class is proportionally represented in both sets. Standardizing the features helps the model learn patterns more effectively, especially for distance-based algorithms like KNN

In [ ]:
# Train KNN
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

# Predict and evaluate KNN
y_pred_knn = knn.predict(X_test)
print("KNN Classification Report:\n", classification_report(y_test, y_pred_knn))
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))

In [ ]:
# Train Logistic Regression
log_reg = LogisticRegression(max_iter=200, C=0.5)
log_reg.fit(X_train, y_train)

# Predict and evaluate Logistic Regression
y_pred_lr = log_reg.predict(X_test)
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_lr))
print("Logistic Regression Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

Logistic Regression, with C=0.5 to balance model regularization, provides a linear approach to classification. Unlike KNN, Logistic Regression finds a decision boundary that separates classes, making it less susceptible to noise in the data. Setting max_iter=200 ensures the model has enough iterations to converge on an optimal solution.

In [ ]:
def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(title)
    plt.show()

# Plot for KNN
cm_knn = confusion_matrix(y_test, y_pred_knn)
plot_confusion_matrix(cm_knn, 'KNN Confusion Matrix')

# Plot for Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)
plot_confusion_matrix(cm_lr, 'Logistic Regression Confusion Matrix')

# Future Considerations

Experiment with Advanced Models: Use an LSTM models that can better capture temporal dependencies in EEG data, as sleep stages naturally transition over time. LSTMs can utilize information from previous time stamps, potentially improving accuracy for stages that are more context-dependent


Address Class Imbalance: The class imbalance in our dataset impacts the model's ability to generalize. We would consider applying weighted loss functions or data augmentation techniques to ensure fairer representation across all sleep stages, which could enhance the model’s accuracy, especially for less-represented stages

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d603abde-6720-4f93-9acc-f0e85e9e8911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>